# Graphs

In [1]:
import numpy as np

## Representing graphs

Graphs are mostly represented as:
- Adjacency list
- Adjacency matrix

### Graphs as Adjacency list

- Each pair in list (x, y) representes the vertex and respective edge weight
- The weight of each edge is taken as 1, making the the graph similar to unweighted

The adjacency list shown below represents an undirected and unweighted graph

In [2]:
adj_list = {
    0: [(1, 1), (2, 1)],
    1: [(0, 1), (3, 1)],
    2: [(0, 1), (3, 1)],
    3: [(1, 1), (2, 1), (4, 1), (5, 1)],
    4: [(3, 1)],
    5: [(3, 1)]
}

### Graphs as Adjacency Matrix

- A node can be reached to itself with 0 weight
- Each edge has a weight 1
- Non adjacent nodes have infinte adjacent distance, indicating no direct connection

The last bit might seem bit unnecessary especially for undirected and unweighted graphs, as a connection can be represented with boolean values of false or true, but including infinite distance allows matrix to encode more data which comes in handy for some algorithms to come

In [3]:
# assuming edge weight is 1
adj_matrix = np.array([[0, 1, 1, np.inf, np.inf, np.inf],
                      [1, 0, np.inf, 1, np.inf, np.inf],
                      [1, np.inf, 0, 1, np.inf, np.inf],
                      [np.inf, 1, 1, 0, 1, 1],
                      [np.inf, np.inf, np.inf, 1, 0, np.inf],
                      [np.inf, np.inf, np.inf, 1, np.inf, 0]])
adj_matrix

array([[ 0.,  1.,  1., inf, inf, inf],
       [ 1.,  0., inf,  1., inf, inf],
       [ 1., inf,  0.,  1., inf, inf],
       [inf,  1.,  1.,  0.,  1.,  1.],
       [inf, inf, inf,  1.,  0., inf],
       [inf, inf, inf,  1., inf,  0.]])

## Converting one representation to other

### Adjacency matrix to Adjacency list

In [4]:
def adjacency_matrix_to_list(adj_matrix):
    
    num_of_vertices = len(adj_matrix)
    adj_list = {}
    
    for vertex in range(num_of_vertices):
        for adj_vertex, adj_vertex_distance in enumerate(adj_matrix[vertex]):
            if 0 < adj_vertex_distance < np.inf:
                adj_list[vertex] = adj_list.get(vertex, []) + [(adj_vertex, adj_vertex_distance)]
    
    return adj_list

adjacency_matrix_to_list(adj_matrix)

{0: [(1, 1.0), (2, 1.0)],
 1: [(0, 1.0), (3, 1.0)],
 2: [(0, 1.0), (3, 1.0)],
 3: [(1, 1.0), (2, 1.0), (4, 1.0), (5, 1.0)],
 4: [(3, 1.0)],
 5: [(3, 1.0)]}

### Adjacency list to Adjacency list

In [5]:
def adjacency_list_to_matrix(adj_list):
    
    num_of_vertices = len(adj_list)
    # initializing all distances to infinity
    adj_matrix = np.full((num_of_vertices, num_of_vertices), np.inf)
    
    for vertex in adj_list:
        
        # setting distance to self as 0
        adj_matrix[vertex, vertex] = 0
        
        for adj_vertex, adj_vertex_distance in adj_list[vertex]:
            
            # setting distance to adjancent nodes to 1 for unweigted graphs
            adj_matrix[vertex, adj_vertex] = adj_vertex_distance
            
    return adj_matrix

adjacency_list_to_matrix(adj_list) 

array([[ 0.,  1.,  1., inf, inf, inf],
       [ 1.,  0., inf,  1., inf, inf],
       [ 1., inf,  0.,  1., inf, inf],
       [inf,  1.,  1.,  0.,  1.,  1.],
       [inf, inf, inf,  1.,  0., inf],
       [inf, inf, inf,  1., inf,  0.]])

## Traversing the graph

Two common approaches to traverse a graph are:

- Breadth First Search (BFS)
- Depth First Search (DFS)

### Breadth First Search

This traversal traverses the graph a level at a time with respect to the start node, meaning first it visits the start, the all adjacent elements to start, them all adjacent element to elements adjacent to start and so on till all elements that can be visited are through the given starting point are visited

Below shown implementation does a breadth first search on a graph represented by an adjacency list

Note: The returned BFS order will only contain the nodes reachable from the start node (its connected component to be precise)

In [6]:
def breadth_first_search(adj_list, start_vertex):
    
    # importing the deque data structure
    from collections import deque
    
    queue = deque()
    
    bfs_order = []
    
    # adding start node to queue
    queue.append(start_vertex)
        
    # iterating while queue is not empty
    while queue:
        # adding first element to the BFS order
        current_vertex = queue.popleft()
        bfs_order.append(current_vertex)
        
        # adding all unvisited adjacent nodes of current_vertex to queue
        for adj_vertex, adj_vertex_distance in adj_list[current_vertex]:
            if adj_vertex not in queue and adj_vertex not in bfs_order:
                queue.append(adj_vertex)
                
    return bfs_order
                

breadth_first_search(adj_list, 0)

[0, 1, 2, 3, 4, 5]

#### Using Breadth First Search to get BFS tree

The result is given as a dictionary with keys reprenting the child nodes and values representing their respective parent nodes.

The obtained tree also gives shortest path from starting vertex in terms of number of edges required to reach the desired vertex

In [7]:
def breadth_first_search_tree(adj_list, start_vertex):
    
    # importing the deque data structure
    from collections import deque
    
    queue = deque()
    
    bfs_order = []
    parent_tree = {start_vertex: start_vertex}
    
    # adding start node to queue
    queue.append(start_vertex)
        
    # iterating while queue is not empty
    while queue:
        # adding first element to the BFS order
        current_vertex = queue.popleft()
        bfs_order.append(current_vertex)
        
        # adding all unvisited adjacent nodes of current_vertex to queue
        for adj_vertex, adj_vertex_distance in adj_list[current_vertex]:
            if adj_vertex not in queue and adj_vertex not in bfs_order:
                queue.append(adj_vertex)
                parent_tree[adj_vertex] = current_vertex
                
    return parent_tree
                

breadth_first_search_tree(adj_list, 0)

{0: 0, 1: 0, 2: 0, 3: 1, 4: 3, 5: 3}

#### Using Breadth First Search to get the edge distance of each traversed vertex

Counting the minimum number of edges required to reach a vertex from starting vertex

In [8]:
def breadth_first_search_with_edge_distances(adj_list, start_vertex):
    
    # importing the deque data structure
    from collections import deque
    
    queue = deque()
    distance_q = deque()
    
    bfs_order = []
    bfs_distance = []
    
    # adding start node to queue and its level to queue
    queue.append(start_vertex)
    distance_q.append(0)
        
    # iterating while queue is not empty
    while queue:
        # adding first element to the BFS order
        current_vertex = queue.popleft()
        edge_distance = distance_q.popleft()
        bfs_order.append(current_vertex)
        bfs_distance.append(edge_distance)
        
        # adding all unvisited adjacent nodes of current_vertex to queue
        for adj_vertex, adj_vertex_distance in adj_list[current_vertex]:
            if adj_vertex not in queue and adj_vertex not in bfs_order:
                queue.append(adj_vertex)
                distance_q.append(edge_distance + 1)
                
    return bfs_order, bfs_distance
                

breadth_first_search_with_edge_distances(adj_list, 0)

([0, 1, 2, 3, 4, 5], [0, 1, 1, 2, 3, 3])

### Depth first search

This method traverses the graph by going to deeper level as far as it can and only backtracking when a "dead end" is reached

Similiar to BFS the implementation uses adjacency list representation and returns a DFS order for the connected component of the start node

In [9]:
def depth_first_search_iterative(adj_list, start_vertex):
    
    stack = []
    dfs_order = []
    
    # adding the start_node to the stack
    stack.append(start_vertex)
    
    # iterating while the stack is not empty
    while stack:
        # adding top of stack to DFS order
        current_vertex = stack.pop()
        dfs_order.append(current_vertex)
        
        # adding all unvisited adjacent vertices of current node to the stack
        for adj_vertex, adj_vertex_distance in adj_list[current_vertex]:
            if adj_vertex not in stack and adj_vertex not in dfs_order:
                stack.append(adj_vertex)
                
    return dfs_order
    
depth_first_search_iterative(adj_list, 0)

[0, 2, 3, 5, 4, 1]

#### Diving deeper in Depth First Search

DFS has an inherent recursive structure which allows for, with some modifications, to do some graph operations efficiently

In [10]:
def depth_first_search(adj_list, start_vertex):
    
    dfs_order = []
    
    def _recurse(vertex):
        # adding vertex to DFS order
        dfs_order.append(vertex)
        
        # finding DFS of unvisited vertices
        for adj_vertex, adj_vertex_distance in adj_list[vertex]:
            if adj_vertex not in dfs_order:
                _recurse(adj_vertex)
                
    _recurse(start_vertex)
    return dfs_order

depth_first_search(adj_list, 0)

[0, 1, 3, 2, 4, 5]

## Shortest path Algorithms

### Dijkstra's Single Source Shortest Path Algorithm

In [11]:
import heapdict

def dijkstra(adj_list, start_node):
    
    # initialize distance to all nodes as infinity
    distances = heapdict.heapdict({vertex: np.inf for vertex in adj_list})
    
    # make distance of start_node as 0
    distances[start_node] = 0
    
    # initialize visited list
    visited = [start_node]
    shortest_distance = [0] 
    
    # initialize parent dictionary
    parent = {start_node: start_node}
    
    while distances:
        current_vertex, current_distance = distances.popitem()
        
        # stop after exploring the connected component containing start node
        if current_distance == np.inf:
            break
        
        for adj_vertex, distance in adj_list[current_vertex]:
            if adj_vertex not in visited and current_distance + distance < distances[adj_vertex]:
                distances[adj_vertex] = current_distance + distance
                parent[adj_vertex] = current_vertex
    
        visited.append(current_vertex)
        shortest_distance.append(current_distance)
    
    return {vertex: dist for vertex, dist in zip(visited, shortest_distance)}, parent

dijkstra(adj_list, 0)

({0: 0, 2: 1, 1: 1, 3: 2, 5: 3, 4: 3}, {0: 0, 1: 0, 2: 0, 3: 2, 4: 3, 5: 3})

To get the shortest path from start we follow the parent pointers returned by parent dictionary

In [12]:
def get_path_from_dict(parent, target):
    
    current = target
    # initialize empty path
    path = []
    
    while parent[current] != current:
        path.append(current)
        current = parent[current]
    
    # adding the parent node
    path.append(current)
    
    return path[::-1]

shortest_distances, parent = dijkstra(adj_list, 0)
get_path_from_dict(parent, 5)

[0, 2, 3, 5]

### Floyd-Warshall All Pairs Shortest Path Algorithm

Unlike Dijkstra's algorithm, Floyd-Warshall's algorithm is well suited for ajdacency matrices

In [13]:
def floyd_warshall(adj_matrix):
    # copying the original matrix to calculate distances
    distances = adj_matrix.copy()
        
    for i in range(len(adj_matrix)):
        for j in range(len(adj_matrix)):
            for k in range(len(adj_matrix)):
                if distances[j,k] > distances[j,i] + distances[i,k]:
                    distances[j,k] = distances[j,i] + distances[i, k]
                    
    return distances

floyd_warshall(adj_matrix)

array([[0., 1., 1., 2., 3., 3.],
       [1., 0., 2., 1., 2., 2.],
       [1., 2., 0., 1., 2., 2.],
       [2., 1., 1., 0., 1., 1.],
       [3., 2., 2., 1., 0., 2.],
       [3., 2., 2., 1., 2., 0.]])

I would love to hear any suggestions or feedback you might have on my notebook. Thanks for reading!